In [1]:
import cv2
import os
import tensorflow as tf
import glob
import numpy as np

In [2]:
TRAIN_IMAGES_PATH = './datasets/images/train'
TEST_IMAGES_PATH = './datasets/images/test'
MODEL_OUTPUT_PATH = './datasets/output/'

In [3]:
def read_captcha_images(image_path):
    if not os.path.exists(image_path):
        raise ValueError('image path is not exist')
        
    images = []
    labels = []
    images_path = os.path.join(image_path, '*.jpg')
    count = 0
    for image_file in glob.glob(images_path):
        count += 1
        if count % 5000 == 0:
            print('Load {} images.'.format(count))
        image = cv2.imread(image_file)
        
        #image = image[:, :, (2, 1, 0)] # change channel
        image = cv2.cvtColor(image, cv2.COLOR_BGR2RGB)
        label = int(image_file.split('_')[-1].split('.')[0])
        images.append(image)
        labels.append(label)
    images = np.array(images)
    labels = np.array(labels)
    return images, labels

In [4]:
def next_batch_set(images, labels, batch_size=128):
    """Generate a batch training data.
    
    Args:
        images: A 4-D array representing the training images.
        labels: A 1-D array representing the classes of images.
        batch_size: An integer.
        
    Return:
        batch_images: A batch of images.
        batch_labels: A batch of labels.
    """
    indices = np.random.choice(len(images), batch_size)
    batch_images = images[indices]
    batch_labels = labels[indices]
    return batch_images, batch_labels

In [5]:
#test_images, test_targets = read_captcha_images(FLAGS.test_images_path)
train_images, train_targets = read_captcha_images(TRAIN_IMAGES_PATH)
test_images, test_targets = next_batch_set(train_images, train_targets)

Load 5000 images.
Load 10000 images.
Load 15000 images.
Load 20000 images.
Load 25000 images.
Load 30000 images.
Load 35000 images.
Load 40000 images.


In [6]:
newInput_X = tf.placeholder(tf.float32, shape=[None, 28, 28, 3])
newInput_dropKeep = tf.placeholder(tf.float32)

In [7]:
def load_graph(frozen_graph_filename):  
    # We parse the graph_def file  
    with tf.gfile.GFile(frozen_graph_filename, "rb") as f:  
        graph_def = tf.GraphDef()  
        graph_def.ParseFromString(f.read())  
  
    # We load the graph_def in the default graph  
    with tf.Graph().as_default() as graph:
        tf.import_graph_def(  
            graph_def,
#             input_map={'inputs:0': newInput_X, 'dropout11/prob:0': newInput_dropKeep},   
#             return_elements=['accuracy/predict:0'], 
            op_dict=None,   
            producer_op_list=None)
        
    return graph

In [8]:
graph = load_graph('./model/captcha_cnn_model.pb')

Instructions for updating:
Please file an issue at https://github.com/tensorflow/tensorflow/issues if you depend on this feature.


In [9]:
for op in graph.get_operations():  
    print(op.name,op.values())

import/inputs (<tf.Tensor 'import/inputs:0' shape=(?, 28, 28, 3) dtype=float32>,)
import/Sub/y (<tf.Tensor 'import/Sub/y:0' shape=() dtype=float32>,)
import/Sub (<tf.Tensor 'import/Sub:0' shape=(?, 28, 28, 3) dtype=float32>,)
import/div/y (<tf.Tensor 'import/div/y:0' shape=() dtype=float32>,)
import/div (<tf.Tensor 'import/div:0' shape=(?, 28, 28, 3) dtype=float32>,)
import/reshape1/Reshape/shape (<tf.Tensor 'import/reshape1/Reshape/shape:0' shape=(4,) dtype=int32>,)
import/reshape1/Reshape (<tf.Tensor 'import/reshape1/Reshape:0' shape=(?, 28, 28, 3) dtype=float32>,)
import/conv1/weight (<tf.Tensor 'import/conv1/weight:0' shape=(3, 3, 3, 32) dtype=float32>,)
import/conv1/weight/read (<tf.Tensor 'import/conv1/weight/read:0' shape=(3, 3, 3, 32) dtype=float32>,)
import/conv1/Variable (<tf.Tensor 'import/conv1/Variable:0' shape=(32,) dtype=float32>,)
import/conv1/Variable/read (<tf.Tensor 'import/conv1/Variable/read:0' shape=(32,) dtype=float32>,)
import/conv1/Conv2D (<tf.Tensor 'import/co

In [10]:
x = graph.get_tensor_by_name('import/inputs:0')
y = graph.get_tensor_by_name('import/accuracy/predict:0')  
          
with tf.Session(graph=graph) as sess:  
    sess.run(tf.global_variables_initializer())
    classes = sess.run(y, feed_dict={x: test_images})
    print(classes)

[5 5 9 6 9 6 0 8 5 0 5 5 6 9 6 0 5 9 9 6 8 9 0 8 3 0 5 9 8 9 8 0 0 6 8 0 0
 9 6 9 0 6 8 5 0 8 0 6 8 9 9 0 0 8 9 9 6 6 6 9 8 5 9 0 6 9 0 0 8 5 9 9 9 6
 8 0 6 9 6 0 9 7 6 8 7 0 5 8 0 5 0 8 9 6 0 6 0 3 9 6 0 9 0 9 0 0 3 8 8 9 9
 5 0 0 9 8 0 9 5 9 0 0 5 6 6 0 6 8]


In [11]:
print (np.mean(np.equal(classes, test_targets)))

0.1015625
